In [18]:
import robin_stocks.robinhood as r 
import pandas as pd
import time
from getpass import getpass

 Setting up Robinhood

In [19]:
username = 'benaltman1@gmail.com'
password = getpass()

login = r.login(username,password)

In [20]:
r.build_holdings()

{'BABA': {'price': '84.970000',
  'quantity': '4.96919200',
  'average_buy_price': '100.6200',
  'equity': '422.23',
  'percent_change': '-15.55',
  'intraday_percent_change': '0.00',
  'equity_change': '-77.767855',
  'type': 'adr',
  'name': 'Alibaba',
  'id': 'b2e06903-5c44-46a4-bd42-2a696f9d68e1',
  'pe_ratio': '21.546400',
  'percentage': '25.72'},
 'MRNA': {'price': '118.680000',
  'quantity': '1.25000400',
  'average_buy_price': '159.9995',
  'equity': '148.35',
  'percent_change': '-25.82',
  'intraday_percent_change': '0.00',
  'equity_change': '-51.649540',
  'type': 'stock',
  'name': 'Moderna',
  'id': '8b760bb0-106d-41ee-a1d5-618236320dd2',
  'pe_ratio': '10.397600',
  'percentage': '9.04'},
 'LYSDY': {'price': '4.755000',
  'quantity': '71.00000000',
  'average_buy_price': '6.7800',
  'equity': '337.61',
  'percent_change': '-29.87',
  'intraday_percent_change': '0.00',
  'equity_change': '-143.775000',
  'type': 'adr',
  'name': 'Lynas Rare Earths',
  'id': '5b647722-739

 Historical Visualization

In [21]:
def extract_list():
    ticker_list = list(r.build_holdings().keys())
    return ticker_list

def visualize_price(ticker_list, span = 'year', bounds = 'regular'):    
    for t in range(len(ticker_list)):
        name = str(r.get_name_by_symbol(ticker_list[t]))
        hist = r.stocks.get_historicals(ticker_list[t], span=span, bounds=bounds)
        hist_df = pd.DataFrame()
        for i in range(len(hist)):
            df = pd.DataFrame(hist[i], index = [i])
            hist_df = pd.concat([hist_df,df])
        hist_df.begins_at = pd.to_datetime(hist_df.begins_at, infer_datetime_format=True)
        hist_df.open_price = hist_df.open_price.astype('float32')
        hist_df.close_price = hist_df.close_price.astype('float32')
        hist_df.high_price = hist_df.high_price.astype('float32')
        hist_df.low_price = hist_df.low_price.astype('float32')

        ax = hist_df.plot(x = 'begins_at', y = 'open_price', figsize = (16,8))
        ax.fill_between(hist_df.begins_at, hist_df.low_price, hist_df.high_price, alpha=0.5)
        ax.set_xlabel('Date')
        ax.set_ylabel('Price (USD)')
        ax.legend([ticker_list[t]])
        ax.set_title(name)

In [22]:
ticker_list = extract_list()
#visualize_price(ticker_list, span = 'year', bounds = 'regular')

 Trading Bot

In [23]:
def trading_bot(trading_dict):
    holdings = r.build_holdings()
    holdings_df = pd.DataFrame()
    for i in range(len(holdings)):
        ticker = list(holdings.items())[i][0]
        holding_df = pd.DataFrame(list(holdings.items())[i][1], index = [i])
        holding_df['ticker'] = ticker
        holdings_df = pd.concat([holdings_df, holding_df])
    holdings_df = holdings_df[['ticker', 'price', 'quantity', 'percent_change','average_buy_price', 'equity', 'equity_change','pe_ratio', 'type', 'name', 'id' ]]

    for j in range(len(trading_dict)):
        holding_df = holdings_df[holdings_df.ticker == list(trading_dict.keys())[j]]
        if holding_df['percent_change'].astype('float32')[0] <= list(trading_dict.values())[j][0]:
            buy_string = 'Buying ' + str(holding_df['ticker'][0]) + ' at ' + time.ctime()
            print(buy_string)
            r.orders.order_buy_market(holding_df['ticker'][0],1,timeInForce= 'gfd')
        else:
            print('Nothing to buy')

        if holding_df['percent_change'].astype('float32')[0] >= list(trading_dict.values())[j][1]:
            sell_string = 'Buying ' + str(holding_df['ticker'][0]) + ' at ' + time.ctime()
            print(sell_string)
            r.orders.order_sell_market(holding_df['ticker'][0],1,timeInForce= 'gfd')
        else:
            print('Nothing to sell')   

In [24]:
trading_dict = {'KMI': [-0.50, 0.50]}
holdings_df = trading_bot(trading_dict)

KeyError: 0

In [ ]:
r.logout()